In [ ]:
### Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

import os

In [ ]:
### Make sure that 'ggplot' style is used for all plots
plt.style.use('ggplot')
# plt.style.available ### To view all other available styles

In [ ]:
### Set Working Directory (WD)
os.chdir('/Volumes/GoogleDrive/My Drive/CEMEX/Data Translators/GitHub/rgamerosl/capstone-project')

In [ ]:
# ### How to import RDS (equivalent to RData) into pandas

# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri

# from rpy2.robjects.conversion import localconverter

# readRDS = robjects.r['readRDS']
# rdata = readRDS('dataset/Fuel_Data.RDS')

# with localconverter(robjects.default_converter + pandas2ri.converter):
#   pdata = robjects.conversion.rpy2py(rdata)

# print(pdata.info())
# display(pdata.head(5))

In [ ]:
### Read the data
df = pd.read_csv("dataset/Fuel_Data.csv")
display(df)

In [ ]:
df.info()

In [ ]:
### Fill with 0 the NA for the different events
df.iloc[:,16:33] = df.iloc[:,16:33].fillna(0)
df.info()

In [ ]:
display(df.head(10))

In [ ]:
df0 = df.drop(['Date','Plate','City','Hrs_eff','Engine_hrs','Fuel_used','km_per_liter'], axis=1)
display(df0.head(10))

In [ ]:
df0.info()

In [ ]:
fig = plt.subplots(figsize=(10,10))
ax = sns.heatmap(df0.iloc[:,0:11].corr(), annot=True, fmt='0.2f', cmap='Blues')
plt.yticks(rotation=0)
# plt.savefig(f'figures/correlations1.png')
plt.show()

### Since Speed and mileage seems to be closely related, probably I should only keep one of them. Sam thing for Trips and Volume

In [ ]:
subset = [7,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
fig = plt.subplots(figsize=(16,16))
ax = sns.heatmap(df0.iloc[:,subset].corr(), annot=True, fmt='0.2f', cmap='Blues',xticklabels=subset,yticklabels=subset)
plt.yticks(rotation=0)
# plt.savefig(f'figures/correlations2.png')
plt.show()

In [ ]:
df1 = df0.dropna(subset=['liters_per_hour'])
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.info()

In [ ]:
# df1.to_excel('dataset/data_v2.xlsx')

In [ ]:
oe_manufacturer = OneHotEncoder()
oe_results_m = oe_manufacturer.fit_transform(df1[['Manufacturer']])
manufacturer_ohe = pd.DataFrame(oe_results_m.toarray(), columns=oe_manufacturer.categories_)
print(display(manufacturer_ohe.head(10)))
manufacturer_ohe.columns=np.array(oe_manufacturer.categories_).flatten()
manufacturer_ohe.info()

In [ ]:
df2 = df1.join(manufacturer_ohe)
### Drop column for Kenworth, before droping it the 29 column corresponded to the manufacturer Kenworth
df2.drop(df2.columns[29],axis=1,inplace=True)
print(display(df2.head(10)))
df2.info()

In [ ]:
### Weekdays (0: Monday to 6: Sunday)
oe_weekday = OneHotEncoder()
oe_results_w = oe_weekday.fit_transform(df2[['Weekday']])
weekday_ohe = pd.DataFrame(oe_results_w.toarray(), columns=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
print(display(weekday_ohe.head(10)))
weekday_ohe.columns=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
weekday_ohe.info()

In [ ]:
df2 = df2.join(weekday_ohe)
### Drop column for Friday, before droping it the 34 column corresponded to the label Friday
df2.drop(df2.columns[34],axis=1,inplace=True)
print(display(df2.head(10)))
df2.info()

In [ ]:
oe_zone = OneHotEncoder()
oe_results_z = oe_zone.fit_transform(df2[['Zone']])
zone_ohe = pd.DataFrame(oe_results_z.toarray(), columns=oe_zone.categories_)
print(display(zone_ohe.head(10)))
zone_ohe.columns=np.array(oe_zone.categories_).flatten()
zone_ohe.info()

In [ ]:
df2 = df2.join(zone_ohe)
### Drop column for PAV, before droping it the 39 column corresponded to the label PAV (Pavimentos)
df2.drop(df2.columns[39],axis=1,inplace=True)
print(display(df2.head(10)))
df2.info()

In [ ]:
# ### Another approach to categorical/indicator variables using get_dummiyes properly
# import pandas as pd

# from pandas.api.types import CategoricalDtype 

# # say you want a column for "japan" too (it'll be always zero, of course)
# df["country"] = train_df["country"].astype(CategoricalDtype(["australia","germany","korea","russia","japan"]))

# # now call .get_dummies() as usual
# pd.get_dummies(df["country"],prefix='country')

In [ ]:
# df2.to_excel("dataset/final_data.xlsx")

In [ ]:
data = df2.drop(['Manufacturer','Zone','Weekday'],axis=1)
data.info()

In [ ]:
data = data.dropna(subset=['Mileage'],axis=0)
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True)
data.info()

In [ ]:
### Now need to do Train Test split and afterwards StandardScale all the numerical variables in each set seperately

In [ ]:
### Train Test split
data_train, data_test = train_test_split(data, test_size=0.25, random_state=42, shuffle=True)

In [ ]:
col_indexes = data.columns[0:23]

In [ ]:
### Standarize numerical variables in Train Set
scaler = StandardScaler()
data_train_scale = data_train.copy(deep=True)
data_train_scale[col_indexes] = scaler.fit_transform(data_train[col_indexes].to_numpy()) 
display(data_train_scale.head(10))

In [ ]:
### Standarize numerical variables in Test Set
scaler = StandardScaler()
data_test_scale = data_test.copy(deep=True)
data_test_scale[col_indexes] = scaler.fit_transform(data_test[col_indexes].to_numpy()) 
display(data_test_scale.head(10))

In [ ]:
X_train = data_train_scale.loc[:, data_train_scale.columns != 'liters_per_hour'].values
y_train = data_train_scale['liters_per_hour'].values

X_test = data_test_scale.loc[:, data_test_scale.columns != 'liters_per_hour'].values
y_test = data_test_scale['liters_per_hour'].values

In [ ]:
def cval_model(X_train,y_train, model='RandomForest', cv=True, LR=0.1):
    '''
    runs crossvalidation in the specified model and returns the MSE and R2 metrics according to the train dataset
    '''
    scores = np.zeros(2)
    if model == 'RandomForest':
        model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=1)
    elif model == 'GradientBoostin':
        model = GradientBoostingRegressor(learning_rate=LR, n_estimators=100, random_state=1)
    else:
        model = AdaBoostRegressor(DecisionTreeClassifier(), learning_rate=LR, n_estimators=100, random_state=1)
    model.fit(X_train, y_train)
    if cv:
        scores[0] = -1*cross_val_score(model, X_train, y_train, scoring= 'neg_mean_squared_error', cv=5).mean()
        scores[1] = cross_val_score(model, X_train, y_train, scoring= 'r2', cv=5).mean()
    return scores

In [ ]:
scores_rf = cval_model(X_train,y_train,model='RandomForest',cv=True)
scores_gdbr = cval_model(X_train,y_train,model='GradientBoostin',cv=True)

print(f'RandomForestRegressor       Train CV    |   MSE: {round(scores_rf[0],3)}   |   R2: {round(scores_rf[1],3)}')
print(f'GradientBoostinRegressor    Train CV    |   MSE: {round(scores_gdbr[0],3)}  |   R2: {round(scores_gdbr[1],3)}')

In [ ]:
### Here I will do a GridSearch to tune the parameters used in the RandomForestRegressor model
random_forest_grid = {'max_depth': [3, None],
                      'max_features': ['sqrt', 'log2', None],
                      'min_samples_split': [2, 4],
                      'min_samples_leaf': [1, 2, 4],
                      'bootstrap': [True, False],
                      'n_estimators': [10, 20, 40, 80],
                      'random_state': [1]}

rf_gridsearch = GridSearchCV(RandomForestRegressor(),
                             random_forest_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='neg_mean_squared_error')

rf_gridsearch.fit(X_train, y_train)

print("best parameters:", rf_gridsearch.best_params_)

best_rf_model = rf_gridsearch.best_estimator_

# Fitting 5 folds for each of 288 candidates, totalling 1440 fits
# [Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
# [Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   42.4s
# [Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  4.9min
# [Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 27.7min
# [Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 80.4min
# [Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 124.7min
# [Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 186.5min finished
# best parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 80, 'random_state': 1}

In [ ]:
### Adjusting Best model to answer the following questions

best_rf = RandomForestRegressor(n_estimators=80, n_jobs=-1, random_state=1, max_features='sqrt',
                                min_samples_leaf=2, min_samples_split=2, max_depth=None, bootstrap=False)
best_rf.fit(X_train, y_train)

In [ ]:
best_rf_y_train_pred = best_rf.predict(X_train)
best_rf_train_MSE_score = mean_squared_error(y_train, best_rf_y_train_pred)
print("MSE for the Best Random Forest in the Train data:", round(best_rf_train_MSE_score,4))
best_rf_train_R2_score = r2_score(y_train, best_rf_y_train_pred)
print("R2 for the Best Random Forest in the Train data:", round(best_rf_train_R2_score,4))

best_rf_y_test_pred = best_rf.predict(X_test)
best_rf_test_MSE_score = mean_squared_error(y_test, best_rf_y_test_pred)
print("MSE for the Best Random Forest in the Test data:", round(best_rf_test_MSE_score,4))
best_rf_test_R2_score = r2_score(y_test, best_rf_y_test_pred)
print("R2 for the Best Random Forest in the Test data:", round(best_rf_test_R2_score,4))

# MSE for the Best Random Forest in the Train data: 0.0352
# R2 for the Best Random Forest in the Train data: 0.9648
# MSE for the Best Random Forest in the Test data: 0.2412
# R2 for the Best Random Forest in the Test data: 0.7588

### Due to the variation between train and test datasets I think there could be some OVERFITTING going on

In [ ]:
importances = best_rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in best_rf.estimators_],
             axis=0)
indices = np.argsort(importances)

# Plot the feature importances of the forest
plt.figure(figsize=(10,10))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")
# If you want to define your own labels,
# change indices to a list of labels on the following line.
plt.yticks(range(X_train.shape[1]), data_train_scale.loc[:, data_train_scale.columns != 'liters_per_hour'].columns[indices[::1]])
plt.ylim([-1, X_train.shape[1]])
plt.savefig(f'figures/feature_importances.png')
plt.show()

In [ ]:
data_train_scale.loc[:, data_train_scale.columns != 'liters_per_hour'].columns[indices[::1]]

In [ ]:
relevant_columns = data_train_scale.loc[:, data_train_scale.columns != 'liters_per_hour'].columns[indices[::-1]][0:21].values.tolist()
relevant_columns.append('liters_per_hour')
new_data = data[relevant_columns]
new_data.info()

In [ ]:
new_data.drop(['Mileage','Volume'],axis=1,inplace=True)
new_data.info()

In [ ]:
### Train Test split
new_data_train, new_data_test = train_test_split(new_data, test_size=0.25, random_state=42, shuffle=True)

In [ ]:
numeric_cols = new_data.columns[[0,1,2,3,4,9,10,11,13,14,15,16,17,18,19]]
print(numeric_cols)

In [ ]:
### Standarize numerical variables in Train Set
scaler = StandardScaler()
new_data_train_scale = new_data_train.copy(deep=True)
new_data_train_scale[numeric_cols] = scaler.fit_transform(new_data_train[numeric_cols].to_numpy()) 
display(new_data_train_scale.head(10))

In [ ]:
### Standarize numerical variables in Test Set
scaler = StandardScaler()
new_data_test_scale = new_data_test.copy(deep=True)
new_data_test_scale[numeric_cols] = scaler.fit_transform(new_data_test[numeric_cols].to_numpy()) 
display(new_data_test_scale.head(10))

In [ ]:
new_X_train = new_data_train_scale.loc[:, new_data_train_scale.columns != 'liters_per_hour'].values
new_y_train = new_data_train_scale['liters_per_hour'].values

new_X_test = new_data_test_scale.loc[:, new_data_test_scale.columns != 'liters_per_hour'].values
new_y_test = new_data_test_scale['liters_per_hour'].values

In [ ]:
new_scores_rf = cval_model(new_X_train,new_y_train,model='RandomForest',cv=True)
new_scores_gdbr = cval_model(new_X_train,new_y_train,model='GradientBoostin',cv=True)

print(f'RandomForestRegressor       Train CV    |   MSE: {round(new_scores_rf[0],3)}   |   R2: {round(new_scores_rf[1],3)}')
print(f'GradientBoostinRegressor    Train CV    |   MSE: {round(new_scores_gdbr[0],3)}  |   R2: {round(new_scores_gdbr[1],3)}')

In [ ]:
### Adjusting Best model to answer the following questions

new_best_rf = RandomForestRegressor(n_estimators=80, n_jobs=-1, random_state=1, max_features='sqrt',
                                min_samples_leaf=2, min_samples_split=2, max_depth=None, bootstrap=False)
new_best_rf.fit(new_X_train, new_y_train)

In [ ]:
new_best_rf_y_train_pred = new_best_rf.predict(new_X_train)
new_best_rf_train_MSE_score = mean_squared_error(new_y_train, new_best_rf_y_train_pred)
print("MSE for the Best Random Forest in the Train data:", round(new_best_rf_train_MSE_score,4))
new_best_rf_train_R2_score = r2_score(new_y_train, new_best_rf_y_train_pred)
print("R2 for the Best Random Forest in the Train data:", round(best_rf_train_R2_score,4))

new_best_rf_y_test_pred = new_best_rf.predict(new_X_test)
new_best_rf_test_MSE_score = mean_squared_error(new_y_test, new_best_rf_y_test_pred)
print("MSE for the Best Random Forest in the Test data:", round(new_best_rf_test_MSE_score,4))
new_best_rf_test_R2_score = r2_score(new_y_test, new_best_rf_y_test_pred)
print("R2 for the Best Random Forest in the Test data:", round(new_best_rf_test_R2_score,4))

In [ ]:
new_importances = new_best_rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in new_best_rf.estimators_],
             axis=0)
indices = np.argsort(new_importances)

# Plot the feature importances of the forest
plt.figure(figsize=(10,10))
plt.title("Feature importances")
plt.barh(range(new_X_train.shape[1]), new_importances[indices],
       color="r", xerr=std[indices], align="center")
# If you want to define your own labels,
# change indices to a list of labels on the following line.
plt.yticks(range(new_X_train.shape[1]), new_data_train_scale.loc[:, new_data_train_scale.columns != 'liters_per_hour'].columns[indices[::1]])
plt.ylim([-1, new_X_train.shape[1]])
plt.savefig(f'figures/new_feature_importances.png')
plt.show()

In [ ]:
### Packages require to adjust Multiple Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [ ]:
X2_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X2_train)
est2 = est.fit()
print(est2.summary())

### I did not expect all features to be relevant according to the p-value (except for 2 or 4 features only: 29, 30, 33, 35)
### Doubt: How to interpret coefficients with non-standarized data?

In [ ]:
print(data.columns)

In [ ]:
y_train_pred_lr = est2.predict(X2_train)
lr_train_MSE_score = mean_squared_error(y_train, y_train_pred_lr)
print("MSE for the Multiple Linear Regression in the Train data:", round(lr_train_MSE_score,4))
lr_train_R2_score = r2_score(y_train, y_train_pred_lr)
print("R2 for the Multiple Linear Regression in the Train data:", round(lr_train_R2_score,4))

X2_test = sm.add_constant(X_test)
y_test_pred_lr = est2.predict(X2_test)
lr_test_MSE_score = mean_squared_error(y_test, y_test_pred_lr)
print("MSE for the Multiple Linear Regression in the Test data:", round(lr_test_MSE_score,4))
lr_test_R2_score = r2_score(y_test, y_test_pred_lr)
print("R2 for the Multiple Linear Regression in the Test data:", round(lr_test_R2_score,4))

In [ ]:
new_X2_train = sm.add_constant(new_X_train)
new_est = sm.OLS(new_y_train, new_X2_train)
new_est2 = new_est.fit()
print(new_est2.summary())

In [ ]:
print(new_data.columns)

In [ ]:
new_y_train_pred_lr = new_est2.predict(new_X2_train)
new_lr_train_MSE_score = mean_squared_error(new_y_train, new_y_train_pred_lr)
print("MSE for the Multiple Linear Regression in the Train data:", round(new_lr_train_MSE_score,4))
new_lr_train_R2_score = r2_score(new_y_train, new_y_train_pred_lr)
print("R2 for the Multiple Linear Regression in the Train data:", round(new_lr_train_R2_score,4))

new_X2_test = sm.add_constant(new_X_test)
new_y_test_pred_lr = new_est2.predict(new_X2_test)
new_lr_test_MSE_score = mean_squared_error(new_y_test, new_y_test_pred_lr)
print("MSE for the Multiple Linear Regression in the Test data:", round(new_lr_test_MSE_score,4))
new_lr_test_R2_score = r2_score(new_y_test, new_y_test_pred_lr)
print("R2 for the Multiple Linear Regression in the Test data:", round(new_lr_test_R2_score,4))

In [ ]:
### Imports require to do the VIF analysis
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif_df = data.iloc[:,0:23]
vif_df.info()

In [ ]:
### First attempt to make this work, but I get some error... still need to look into it with more details
features = "+".join(vif_df.columns - ["liters_per_hour"])
vif_y, vif_X = dmatrices('liters_per_hour ~' + features, df, return_type='dataframe')

vif_model = pd.DataFrame()
vif_model["VIF Factor"] = [variance_inflation_factor(vif_X.values, i) for i in range(vif_X.shape[1])]
vif_model["features"] = vif_X.columns